In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# Initialize device as "mps", or the GPU, if it's available; else, use CPU
device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Initial block size
block_size: int = 8
# Initial batch size
batch_size: int = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [25]:
# Creates the sorted list of characters and vocabular size
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text: str = f.read()
chars: list[str] = sorted(set(text))
print(chars)
vocab_size: int = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [26]:
# Initialize dictionaries for characters and their associated tokens
string_to_int: dict[str, int] = { ch:i for i,ch in enumerate(chars) }
int_to_string: dict[int, str] = { i:ch for i,ch in enumerate(chars) }

# Define lambda functoins to convert characters to tokens and vice versa
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

# Creates a Tensor of int-64s out of the tokenized characters
data: torch.Tensor = torch.tensor(encode(text), dtype=torch.long)

In [31]:
# Initializes the training data and validation data
n: int = int(0.8*len(data))
train_data: torch.Tensor = data[:n]
val_data: torch.Tensor = data[n:]

def get_batch(split: str) -> tuple[torch.Tensor, torch.Tensor]:
    """Gets a Tensor (batches of blocks) from the specified data set

    This method grabs either the training or validation data set and creates
    a Tensor from it with the block_size and batch_size values.

    Args:
        split (str): The train or val split

    Returns:
        tuple[torch.Tensor, torch.Tensor]: The inputs and their target parameters
    """
    data: torch.Tensor = train_data if split == 'train' else val_data
    ix: torch.Tensor = torch.randint(len(data) - block_size, (batch_size,))
    x: torch.Tensor = torch.stack([data[i:i+block_size] for i in ix])
    y: torch.Tensor = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [36]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [37]:
class BigramLanguageModel(nn.Module):
    """The Bigram Language Model

    Extends:
        nn.Module (class): The nn machine learning subclass provided by PyTorch

    Methods:
        __init__(vocab_size: int) -> None
        forward(index, targets)
    """
    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_embedding_table: nn.Embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range (max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


d
"*U"LDqEf_M
giOI(T]VB'eH)zl?pIRW!Fpwa,P﻿2(,F5g5vFeF[X2Cv&C5RD0mx﻿PnDFn)k?UdrGYRUFm.mj:g0_RShT88MIH]aNH!?ZWOqe)]sH)j9A)pivrj
X;Svv*Z
Rb?x]td&r16720Hg CPT";eFUSAVK*DD, e_Q(,F;JbbxNL0
k3Ug&TDX2*b
IIzlF;w'ywxr﻿f?iOGiuS
)Zumr:NPv?U3zU7)L(y5:p9
lVKHWV*!Qw_[_e_rD1[W_C 51)dNuWbX﻿:[Rn
qzkCxy6]L]gF﻿4czQUX:i.(!mfr9J2eF[sk5q[Wv: er)sKZ:6P[_vF
0shKR4X9TiWv8&unvX2l uY.]YN-H)p7gArpI0aqlxZ'js8eeFg8vAPnxFblTiiiOeAUXeV_BTzxjg '*Nu)1?(kjdBeB&jfC0Fbb
0WIBeMdHT._We!P[_YR'T.AmrDJILwfE[qK4o2 ?!sZ1PeY6.
7H5fTbGH﻿mOmz


In [43]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'Step: {iter} | Train Loss: {losses['train']:.4f} | Val Loss: {losses['val']:.4f}')
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

Step: 0 | Train Loss: 2.6290 | Val Loss: 2.6892
Step: 250 | Train Loss: 2.6453 | Val Loss: 2.6466
Step: 500 | Train Loss: 2.6159 | Val Loss: 2.6742
Step: 750 | Train Loss: 2.6384 | Val Loss: 2.6663
Step: 1000 | Train Loss: 2.6063 | Val Loss: 2.6477
Step: 1250 | Train Loss: 2.6198 | Val Loss: 2.6477
Step: 1500 | Train Loss: 2.6239 | Val Loss: 2.6716
Step: 1750 | Train Loss: 2.6048 | Val Loss: 2.6341
Step: 2000 | Train Loss: 2.6238 | Val Loss: 2.6238
Step: 2250 | Train Loss: 2.6003 | Val Loss: 2.6143
Step: 2500 | Train Loss: 2.6124 | Val Loss: 2.6376
Step: 2750 | Train Loss: 2.5919 | Val Loss: 2.6475
Step: 3000 | Train Loss: 2.5808 | Val Loss: 2.6433
Step: 3250 | Train Loss: 2.5825 | Val Loss: 2.6342
Step: 3500 | Train Loss: 2.5823 | Val Loss: 2.6145
Step: 3750 | Train Loss: 2.5686 | Val Loss: 2.5938
Step: 4000 | Train Loss: 2.5831 | Val Loss: 2.5951
Step: 4250 | Train Loss: 2.5379 | Val Loss: 2.5984
Step: 4500 | Train Loss: 2.5738 | Val Loss: 2.5723
Step: 4750 | Train Loss: 2.5576 | Val

In [34]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"BDI'P1
ZesNd mBfbfyEd;Z5Y0j!?W49(d  trd﻿
vh!:imit5S5zeKed;y'P?:t4H]S,w_caghete'&a fOjB1d.NK9Y)Tbedidl;.) ug. fozthlTdd fi&DDYbe.
Eoqwd,Ces5hZZ1fy1WLYurdgvegBIn8qkpDSn[UO!:Q*8.
&Jck-. XVfF2;;oO,&CAWki]K5J m&64r_EmU"to5_ZZCK; fGNl. bVuZ2N4HrgP?Ogxv("X7E*1iv88z?:z2,"?v﻿ajuenB1hyrW8n]P[NG; Dd.?1 TbA
T_uz7qvonQvet?mid,
U﻿(Vnd C_tr l[L3HRKNQEyt-V﻿&
"2C,b6O[xDDw"W];gjWW7x6CL g﻿Q'DqW)ia.)an-Gitlz8G4scY

Di'Ljune!6'duit pC(1piP
v!JU-V Nx,J6]gt7tXor0Ny
piPl.A.&:taj!r,[( d::

chevZI612CA:F:I9)Wuid'S'nheep
